In [105]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [106]:
(x_train,y_train),(x_val,y_val) = keras.datasets.cifar10.load_data()
print(x_train.shape)
print(y_train.shape)

(50000, 32, 32, 3)
(50000, 1)


In [107]:
#自定义残差块
#每个残差块包括两个卷积层
class BasicBlock(keras.Model):
    def __init__(self,filter_num,stride = 1):
        super(BasicBlock,self).__init__()
        #padding为same保证输出的size为输入的size整除strides后得到的
        self.conv1 = keras.layers.Conv2D(filters=filter_num,kernel_size=(3,3),strides=stride,padding='same')
        self.bn1 = keras.layers.BatchNormalization()
        
        self.conv2 = keras.layers.Conv2D(filters=filter_num,kernel_size=(3,3),strides=1,padding='same')
        self.bn2 = keras.layers.BatchNormalization()
        
        #残差块的输入经过卷积层以后，输出的size可能发生改变，为了保证size一致从而能相加，
        if stride!=1:#当步长不为1时，size改变
            self.downsample = keras.Sequential()
            self.downsample.add(keras.layers.Conv2D(filters=filter_num,kernel_size=(1,1),strides=stride))
        else:#如果为1，size没有改变，原样输出
            self.downsample = lambda x:x
    
    def call(self,inputs,traing=None):
        out = self.conv1(inputs)
        out = self.bn1(out)
        out = keras.activations.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        
        identity = self.downsample(inputs)#调整维度
        
        out = keras.layers.add([out,identity])#恒等映射
        
        return keras.activations.relu(out)

In [108]:
def ResNet18():
    input = keras.layers.Input(shape=[32,32,3])
    x = tf.keras.layers.Conv2D(64, 3 , strides=1, padding='same', activation="relu",kernel_initializer=initializers.he_uniform())(input)
    x = tf.keras.layers.BatchNormalization()(x)
    for filter_num in [64,64,128,128,256,256,512,512]:#8个残差块，每个残差块2个卷积，一共16层
        x = BasicBlock(filter_num,2)(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = layers.Flatten()(x)
    out = layers.Dense(10, activation=tf.nn.softmax)(x)
    model = keras.models.Model(inputs=input, outputs=out)
    return model

In [109]:
model = ResNet18()

In [110]:
model.compile(optimizer=keras.optimizers.Adam(0.01),loss=tf.losses.sparse_categorical_crossentropy,metrics=['accuracy'])

In [111]:
#加入数据增强
datagen = ImageDataGenerator( 
            rotation_range=20, #图片随机转动的角度
            width_shift_range=0.2, #图片水平偏移的幅度
            height_shift_range=0.2, #图片竖直偏移的幅度
            horizontal_flip=True,#进行随机水平翻转
            validation_split=0.13)# 保留用于验证的图像的比例（严格在0和1之间)

In [112]:
model.fit_generator(datagen.flow(x_train,y_train,batch_size=64),
                    validation_data=(x_val,y_val),validation_freq=1,
                    epochs=50,
                    callbacks=[keras.callbacks.ReduceLROnPlateau()])

Epoch 1/50
782/782 [==============================] - 109s 139ms/step - loss: 1.9779 - accuracy: 0.2492 - val_loss: 2.9492 - val_accuracy: 0.1593
Epoch 2/50
782/782 [==============================] - 81s 104ms/step - loss: 1.7015 - accuracy: 0.3540 - val_loss: 1.9921 - val_accuracy: 0.3285
Epoch 3/50
782/782 [==============================] - 82s 105ms/step - loss: 1.5588 - accuracy: 0.4155 - val_loss: 1.5995 - val_accuracy: 0.4273
Epoch 4/50
782/782 [==============================] - 83s 106ms/step - loss: 1.4485 - accuracy: 0.4653 - val_loss: 1.3968 - val_accuracy: 0.4922
Epoch 5/50
782/782 [==============================] - 83s 106ms/step - loss: 1.3316 - accuracy: 0.5197 - val_loss: 1.3762 - val_accuracy: 0.5234
Epoch 6/50
782/782 [==============================] - 83s 106ms/step - loss: 1.2489 - accuracy: 0.5551 - val_loss: 1.5620 - val_accuracy: 0.4893
Epoch 7/50
782/782 [==============================] - 83s 106ms/step - loss: 1.1768 - accuracy: 0.5863 - val_loss: 1.6537 - val_a

In [113]:
model.evaluate(x_val,y_val)

10000/10000 [==============================] - 6s 568us/sample - loss: 0.6221 - accuracy: 0.7904


[0.6221348830938339, 0.7904]